# Generate qsub scripts to compute mean and std per variable

In [ ]:
import os
import sys
import yaml
import numpy as np
import xarray as xr

In [ ]:
sys.path.insert(0, os.path.realpath('../libs/'))
import preprocess_utils as pu

## 6 hourly mean std

In [ ]:
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [ ]:
varnames = list(conf['residual'].keys())
varnames = varnames[:-3] # remove save_loc and others

In [ ]:
varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q']))
varname_upper = ['U', 'V', 'T', 'Q']

In [ ]:
for i_var, varname in enumerate(varname_surf):
    
    f = open('{}residual_6h_{}.sh'.format(conf['qsub']['qsub_loc'], varname), 'w') 
    
    heads = '''#!/bin/bash -l

#PBS -N {}
#PBS -A NAML0001
#PBS -l walltime=23:59:59
#PBS -l select=1:ncpus=4
#PBS -q casper
#PBS -o residual_{}.log
#PBS -e residual_{}.err

conda activate credit
cd {}
python residual_6h.py {} {}
'''.format(varname, varname, varname, 
           conf['qsub']['scripts_loc'], 
           varname,
           'None')
    
    print(heads, file=f)    
    f.close()

f = open('{}preprocess_surf_residual_6h.sh'.format(conf['qsub']['qsub_loc']), 'w')

for i_var, varname in enumerate(varname_surf):
    print('qsub residual_6h_{}.sh'.format(varname), file=f)
    
f.close()

In [ ]:
for i_var, varname in enumerate(varname_upper):
    for i_level in range(15):
        
        f = open('{}residual_6h_{}_level{}.sh'.format(conf['qsub']['qsub_loc'], varname, i_level), 'w') 
        
        heads = '''#!/bin/bash -l
    
#PBS -N {}
#PBS -A NAML0001
#PBS -l walltime=23:59:59
#PBS -l select=1:ncpus=4
#PBS -q casper
#PBS -o residual_{}.log
#PBS -e residual_{}.err

conda activate credit
cd {}
python residual_6h.py {} {}
'''.format(varname, varname, varname, 
           conf['qsub']['scripts_loc'], 
           varname,
           i_level)
        
        print(heads, file=f)    
        f.close()

f = open('{}preprocess_upper_residual_6h.sh'.format(conf['qsub']['qsub_loc']), 'w')

for i_var, varname in enumerate(varname_upper):
    for i_level in range(15):
        print('qsub residual_6h_{}_level{}.sh'.format(varname, i_level), file=f)
    
f.close()